## modelos de texto, dados e treinamento

In [ ]:
from fastai.gen_doc.nbdoc import *

O módulo [`text`](/text.html#text) da biblioteca fastai contém todas as funções necessárias para definir um conjunto de dados apropriado para as diversas tarefas NLP (Natural Language Processing) e rapidamente gerar modelos que você pode usar para eles. Especificamente:
- [`text.transform`](/text.transform.html#text.transform) contém todos os scripts para pré-processar seus dados, a partir de texto simples ao token ids,
- [`text.data`](/text.data.html#text.data) contém a definição de [`TextDataBunch`](/text.data.html#TextDataBunch), que a classe principal que você precisa em PNL,
- [`text.learner`](/text.learner.html#text.learner) contém funções auxiliares para criar rapidamente um modelo de linguagem ou um classificador RNN.
Dê uma olhada nos links acima para obter mais detalhes da API de cada módulo, de ler para uma visão geral rápida.

## Início Rápido: Treinando um modelo sentimento IMDb com * ULMFiT *

Vamos começar com uma rápida exemplo end-to-end de treinamento de um modelo. Vamos treinar um classificador sentimento em uma amostra dos dados populares IMDb, mostrando 4 etapas:
1. Leitura e visualização dos dados IMDb
1. Começar seus dados prontos para modelagem
1. Ajustar um modelo de linguagem
1. Construir um classificador

### Leitura e visualização dos dados da IMDb

Primeiro vamos importar tudo o que precisamos para o texto.

In [ ]:
from fastai.text import * 

Ao contrário do que imagens em Computer Vision, texto pode não ser directamente transformada em números a ser alimentado em um modelo. A primeira coisa que precisamos fazer é pré-processar os dados para que mudar os textos brutos para listas de palavras ou tokens (um passo que é chamado tokenization), em seguida, transformar essas fichas em números (um passo que é chamado numericalization). Estes números são, então, passados ​​para a incorporação de camadas que vai convertê-los em matrizes de flutuadores antes de passá-los através de um modelo.
Você pode encontrar na web abundância de [Word Embeddings](https://en.wikipedia.org/wiki/Word_embedding) para converter diretamente suas fichas em carros alegóricos. Essas incorporações palavra têm geralmente ser treinado em um grande corpus como a wikipedia. Na sequência dos trabalhos de [ULMFiT](https://arxiv.org/abs/1801.06146), a biblioteca fastai é mais focada sobre o uso pré-treinados Modelos de Linguagem e afinar-los. embeddings Word são apenas vetores de 300 ou 400 carros alegóricos que representam palavras diferentes, mas um modelo de linguagem pré-treinado não só tem aqueles, mas também foi treinado para obter uma representação de frases completas e documentos.
É por isso que a biblioteca está estruturada em torno de três passos:
1. obter os seus dados pré-processados ​​e prontos para usar em uma quantidade mínima de código,
1. Criar um modelo de linguagem com pesos pré-treinado que você pode ajustar para o seu conjunto de dados,
1. Crie outros modelos, como classificadores em cima do codificador do modelo de linguagem.
Para mostrar exemplos, nós fornecemos uma pequena amostra do [IMDB dataset](https://www.imdb.com/interfaces/) que contém 1.000 comentários de filmes com etiquetas (positivos ou negativos).

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

Criando um conjunto de dados a partir de seus textos crus é muito simples se você tê-lo em uma dessas maneiras
- organizou em pastas com um estilo IMAGEnet
- organizada em um arquivo CSV com as colunas rótulos e um colunas de texto
Aqui, a amostra de title está em um arquivos de textos CSV que se parece com isso:

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


### Obtendo seus dados prontos para modelagem

In [ ]:
for file in ['train_tok.npy', 'valid_tok.npy']:
    if os.path.exists(path/'tmp'/file): os.remove(path/'tmp'/file)

Para obter um [`DataBunch`](/basic_data.html#DataBunch) rapidamente, há também vários métodos de fábrica, dependendo de como os nossos dados é estruturado. Eles estão todos detalhados no [`text.data`](/text.data.html#text.data), aqui vamos usar o método <code> from_csv </ code> do [`TextLMDataBunch`](/text.data.html#TextLMDataBunch) (para obter os dados prontos para um modelo de linguagem) e [`TextClasDataBunch`](/text.data.html#TextClasDataBunch) (para obter os dados prontos para um classificador de texto) classes .

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

Isto faz todo o pré-processamento necessário por trás da cena. Para o classificador, nós também passar o vocabulário (mapeamento de ids para palavras) que deseja usar: isso é para garantir que `data_clas` usará o mesmo dicionário como` data_lm`.
Uma vez que este passo pode ser um pouco demorado, é melhor para salvar o resultado com:

In [ ]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

Isto irá criar um diretório 'tmp', onde todo o material computadorizada será armazenado. Você pode então recarregar esses resultados com:

In [ ]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

Note que você pode carregar os dados com diferentes parâmetros [`DataBunch`](/basic_data.html#DataBunch) (tamanho do lote, `bptt`, ...)

### Ajustar um modelo de linguagem

Podemos usar o `data_lm` objeto criado anteriormente para afinar um modelo de linguagem pré-treinado. [fast.ai](http://www.fast.ai/) tem um modelo de Inglês com uma arquitetura AWD-LSTM disponível que pode baixar. Podemos criar um objeto aluno que irá criar diretamente um modelo, baixar os pesos pré-treinado e estar pronto para o ajuste fino.

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.319174,3.882361,0.288155,00:13


Como um modelo de visão por computador, podemos descongelar o modelo e ajustar-lo.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.935607,3.811058,0.297262,00:16


Para avaliar o seu modelo de linguagem, você pode executar o método [`Learner.predict`](/basic_train.html#Learner.predict) e especificar o número de palavras que você deseja que ele adivinhar.

In [ ]:
learn.predict("This is a review about", n_words=10)

'This is a review about what was worth a word of reading , where some'

Não faz muito sentido (temos uma pequena vocabulário aqui e não treinou muito sobre ele), mas note que respeita gramática básica (que vem do modelo pré-treinado).
Finalmente, salve o codificador para ser capaz de usá-lo para a classificação na próxima seção.

In [ ]:
learn.save_encoder('ft_enc')

### Construção de um classificador

Vamos agora usar o `data_clas` objeto criado anteriormente para construir um classificador com nosso codificador afinado. O objeto aluno pode ser feito em uma única linha.

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [ ]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative


In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.603350,0.531586,0.741294,00:23


Mais uma vez, podemos descongelar o modelo e ajustar-lo.

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.503833,0.444084,0.800995,00:29


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.410422,0.363226,0.850746,00:42


Mais uma vez, podemos prever em um texto simples usando o método [`Learner.predict`](/basic_train.html#Learner.predict).

In [ ]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.0049, 0.9951]))